In [192]:
# import requests
# import csv
# import json
# import time
# from datetime import datetime, timedelta
# from collections import defaultdict

# api_key = 'mzrsaV5E9ZwVPQ2biOQGyYansI7Ukh5p'
# api_key = 'CGeuqVOhGnPEhw8fZaxDDpSXMklkLVJo'


In [193]:
# def print_json(json_data):
#     print(json.dumps(json_data, indent=2))

# def write_jsonl_file(data, filename):
#     # Write data to a JSONL file
#     with open(filename, 'a', encoding='utf-8') as file:
#         json.dump(data, file, ensure_ascii=False)
#         file.write('\n')


In [194]:
# def return_response(url) :
#     response = requests.get(url)
#     return response.json()

# def generate_date_range(start_date, end_date):
#     # Generate a date range between start_date and end_date
#     current_date = start_date
#     while current_date <= end_date:
#         yield current_date
#         current_date += timedelta(weeks=1)


# Getting All Lists

In [195]:
# output_filename = 'all_lists.json'
# with open(output_filename, 'w', encoding='utf-8') as file:
#     url = f'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={api_key}' # all lists
#     response = return_response(url)

#     json.dump(response, file, indent=4)
#     print(f'All lists saved to {output_filename}.')


# Getting All Weeks

In [196]:
# start_date_str = '2011-02-13'
# start_date_str = '2017-01-01' # API breakpoint
# end_date_str = '2023-12-10'

# start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
# end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

# all_responses = {}
# failed_week = []

# output_filename = 'all_weeks_2.json'
# with open(output_filename, 'w', encoding='utf-8') as file:
#     for current_date in generate_date_range(start_date, end_date):
#         time.sleep(12) #
#         week = current_date.strftime('%Y-%m-%d')
#         url = f'https://api.nytimes.com/svc/books/v3/lists/{week}/combined-print-and-e-book-fiction.json?api-key={api_key}'
#         response = return_response(url)

#         if ('status' in response and response['status']=='OK'):
#             book = response['results']['books'][0]
#             book = {
#                 'date': response['results']['published_date'],
#                 'rank': book['rank'],
#                 'title': book['title'],
#                 'author': book['author'],
#                 'primary_isbn10': book['primary_isbn10'],
#                 'dagger': book['dagger'],
#                 'publisher': book['publisher'],
#                 'weeks_on_list': book['weeks_on_list']
#             }
#             month = book['date'].split("-")[0]+'-'+book['date'].split("-")[1]
#             if month in all_responses:
#                 all_responses[month].append(book)
#             else:
#                 all_responses[month] = [book]

#             print(f'Response for week {week} added to the list.')

#         else:
#             print(week)
#             failed_week.append(week)

#     json.dump(all_responses, file, indent=2)


# Parsing data into months

In [197]:
# from collections import Counter

# all_responses_month = {}

# input_filename = 'all_weeks.json'
# output_filename = 'all_months.json'
# with open(output_filename, 'w', encoding='utf-8') as file:
#     with open(input_filename, 'r', encoding='utf-8') as read_file:
#         data = json.load(read_file)

#         for month in data.keys():
#             book_list = data[month]

#             titles = [book['title'] for book in book_list]
#             title_counts = Counter(titles)
#             most_common_title = title_counts.most_common(1)[0][0]

#             most_common_book = next(book for book in book_list if book['title'] == most_common_title)
#             all_responses_month[month] = most_common_book 
            
#     json.dump(all_responses_month, file, indent=2)


In [198]:
# import csv
# from collections import Counter
# import json

# all_responses_month = {}

# input_filename = 'all_weeks.json'
# output_filename = 'p3js/all_months.csv'

# with open(output_filename, 'w', encoding='utf-8', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file)
#     header = ['rank', 'title', 'author','primary_isbn10','dagger','publisher','weeks_on_list']
#     csv_writer.writerow(['month']+header)  # Add other column headers as needed

#     with open(input_filename, 'r', encoding='utf-8') as read_file:
#         data = json.load(read_file)

#         for month in data.keys():
#             book_list = data[month]

#             titles = [book['title'] for book in book_list]
#             title_counts = Counter(titles)
#             most_common_title = title_counts.most_common(1)[0][0]

#             most_common_book = next(book for book in book_list if book['title'] == most_common_title)
#             all_responses_month[month] = most_common_book

#             row = [most_common_book[key] for key in header]
#             csv_writer.writerow([month]+row)  # Add other column values as needed


In [199]:
# import csv
# import json

# # Paths to your files
# json_input_filename = 'all_weeks.json'
# csv_input_filename = 'all_months.csv'
# output_filename = 'all_weeks_updated.json'

# # Load the original JSON data
# with open(json_input_filename, 'r', encoding='utf-8') as json_file:
#     original_data = json.load(json_file)

# # Create a dictionary to store additional details based on book title
# book_details = {}

# # Read the detailed information from the CSV file
# with open(csv_input_filename, 'r', encoding='utf-8') as csv_file:
#     csv_reader = csv.DictReader(csv_file)

#     for row in csv_reader:
#         title = row['title']
#         genre = row['genre']
#         pages = row['pages']

#         # Store genre and page information based on book title
#         book_details[title] = {'genre': genre, 'pages': pages}

# # Update the original data with additional details
# for month, books in original_data.items():
#     for book in books:
#         title = book['title']

#         # Check if additional details are available for this book title
#         if title in book_details:
#             book['genre'] = book_details[title]['genre']
#             book['pages'] = book_details[title]['pages']

# # Save the updated data to a new JSON file
# with open(output_filename, 'w', encoding='utf-8') as json_output_file:
#     json.dump(original_data, json_output_file, indent=2)

# print(f"Updated data saved to {output_filename}")


Updated data saved to all_weeks_updated.json


In [200]:
# import csv
# import json
# from collections import Counter

# all_responses_month = {}

# input_filename = 'all_weeks_updated.json'
# output_filename = 'all_months_again.csv'

# # Open the output CSV file for writing
# with open(output_filename, 'w', encoding='utf-8', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file)
    
#     # Specify the header for the CSV file
#     header = ['rank', 'title', 'author', 'primary_isbn10', 'dagger', 'publisher', 'weeks_on_list','genre','pages']
    
#     csv_writer.writerow(['month'] + header)  # Add other column headers as needed

#     with open(input_filename, 'r', encoding='utf-8') as read_file:
#         data = json.load(read_file)

#         all_books = [book for month_data in data.values() for book in month_data]

#         title_counts = Counter(book['title'] for book in all_books)
#         author_counts = Counter(book['author'] for book in all_books)

#         for month in data.keys():
#             book_list = data[month]
#             book_list.reverse() # use highest week count
            
#             most_common_book = max(book_list, key=lambda book: title_counts[book['title']])
#             if title_counts[most_common_book['title']] == 1: ## if same, use author count
#                 max_weeks_on_list = max(book_list, key=lambda book: book['weeks_on_list'])['weeks_on_list']
#                 most_common_books = [book for book in book_list if book['weeks_on_list'] == max_weeks_on_list]

#                 if len(most_common_books) == 1:
#                     most_common_book = most_common_books[0]
#                 else:
#                     most_common_book = max(book_list, key=lambda book: author_counts[book['author']])
#                 all_responses_month[month] = most_common_book
#             print(most_common_book['title'])

#             row = [most_common_book[key] for key in header]
#             csv_writer.writerow([month]+row)  



TICK TOCK
WATER FOR ELEPHANTS
WATER FOR ELEPHANTS
WATER FOR ELEPHANTS
THE HELP
THE HELP
THE HELP
THE HELP
THE HELP
11/22/63
THE DROP
THE HELP
EXTREMELY LOUD AND INCREDIBLY CLOSE
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
REFLECTED IN YOU
THE RACKETEER
GONE GIRL
SAFE HAVEN
SAFE HAVEN
SAFE HAVEN
THE BET
WHISKEY BEACH
INFERNO
INFERNO
THE CUCKOO'S CALLING
THE CUCKOO'S CALLING
STORM FRONT
SYCAMORE ROW
SYCAMORE ROW
SYCAMORE ROW
THE GOLDFINCH
CONCEALED IN DEATH
SHADOW SPELL
UNLUCKY 13
UNLUCKY 13
INVISIBLE
FIFTY SHADES OF GREY
OUTLANDER
GONE GIRL
GONE GIRL
ALL THE LIGHT WE CANNOT SEE
GONE GIRL
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
GREY
GO SET A WATCHMAN
MAKE ME
THE MARTIAN
ROGUE LAWYER
ROGUE LAWYER
THE GIRL ON THE TRAIN
BROTHERHOOD IN DEATH
ME BEFORE YOU
FOOL ME ONCE
ME BEFORE YOU
ME BEFORE YOU
ME BEFORE YOU
THE GIRL ON 

In [206]:
import csv
import json
from collections import Counter

all_responses_month = {}

input_filename = 'all_weeks_updated.json'
output_filename = 'all_months.json'

with open(input_filename, 'r', encoding='utf-8') as read_file:
    data = json.load(read_file)

    all_books = [book for month_data in data.values() for book in month_data]

    title_counts = Counter(book['title'] for book in all_books)
    author_counts = Counter(book['author'] for book in all_books)

    for month in data.keys():
        book_list = data[month]
        book_list.reverse() # use highest week count
        
        most_common_book = max(book_list, key=lambda book: title_counts[book['title']])
        if title_counts[most_common_book['title']] == 1: ## if same, use author count
            max_weeks_on_list = max(book_list, key=lambda book: book['weeks_on_list'])['weeks_on_list']
            most_common_books = [book for book in book_list if book['weeks_on_list'] == max_weeks_on_list]

            if len(most_common_books) == 1:
                most_common_book = most_common_books[0]
            else:
                most_common_book = max(book_list, key=lambda book: author_counts[book['author']])
            all_responses_month[month] = most_common_book
        print(most_common_book['title'])

        all_responses_month[month] = most_common_book

with open(output_filename, 'w', encoding='utf-8') as json_output_file:
    json.dump(all_responses_month, json_output_file, indent=2)


TICK TOCK
WATER FOR ELEPHANTS
WATER FOR ELEPHANTS
WATER FOR ELEPHANTS
THE HELP
THE HELP
THE HELP
THE HELP
THE HELP
11/22/63
THE DROP
THE HELP
EXTREMELY LOUD AND INCREDIBLY CLOSE
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
FIFTY SHADES OF GREY
REFLECTED IN YOU
THE RACKETEER
GONE GIRL
SAFE HAVEN
SAFE HAVEN
SAFE HAVEN
THE BET
WHISKEY BEACH
INFERNO
INFERNO
THE CUCKOO'S CALLING
THE CUCKOO'S CALLING
STORM FRONT
SYCAMORE ROW
SYCAMORE ROW
SYCAMORE ROW
THE GOLDFINCH
CONCEALED IN DEATH
SHADOW SPELL
UNLUCKY 13
UNLUCKY 13
INVISIBLE
FIFTY SHADES OF GREY
OUTLANDER
GONE GIRL
GONE GIRL
ALL THE LIGHT WE CANNOT SEE
GONE GIRL
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
THE GIRL ON THE TRAIN
GREY
GO SET A WATCHMAN
MAKE ME
THE MARTIAN
ROGUE LAWYER
ROGUE LAWYER
THE GIRL ON THE TRAIN
BROTHERHOOD IN DEATH
ME BEFORE YOU
FOOL ME ONCE
ME BEFORE YOU
ME BEFORE YOU
ME BEFORE YOU
THE GIRL ON 

AUTHOR INFO

In [ ]:
import csv
import json

input_filename = 'all_months_author_added.csv'
output_filename = 'author_info.json'

# Initialize an empty dictionary to store author data
author_data = {}

# Open the CSV file for reading
with open(input_filename, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Iterate through each row in the CSV
    for row in csv_reader:
        # Extract relevant information from the row
        author_names = row['author'].split(' and ')

        author_gender = row['author_gender']
        book_title = row['title']
        
        for author_name in author_names:
            if author_name not in author_data:
                author_data[author_name] = {'gender': author_gender, 'books': []}
            
            # Add the book to the author's list of books
            if book_title not in author_data[author_name]['books']:
                author_data[author_name]['books'].append(book_title)

# Save the author data to a JSON file
with open(output_filename, 'w', encoding='utf-8') as json_file:
    json.dump(author_data, json_file, indent=2)
